# Minesweeper
---
_**Module**: HSLU - Artificial Intelligence - Search & Optimization (AISO)_  
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Adrian Kauz_

>The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines 💣. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines!

## Imports

In [2]:
from ortools.constraint_solver import pywrapcp
import numpy as np

## Helper functions

In [3]:
def printSolution(currentSolution):
    print("Possible solution:")
    
    for row in range(len(currentSolution)):
        currentRow = ""
        for column in range(len(currentSolution[row])):
            currentRow += "{} ".format(currentSolution[row][column].Value())
        
        print(currentRow)

## Solution

In [11]:
solver = pywrapcp.Solver("MineSweeper")

_ = None

playground = [
    [1, _, _, _, 2, _, _, 1],
    [_, 1, 2, _, 3, _, _, 1],
    [_, 2, _, 1, _, _, _, 0],
    [_, 2, 1, _, _, 2, 3, 1],
    [_, _, _, 2, _, _, _, _],
    [1, _, _, _, 4, 3, _, _],
    [_, 1, _, _, 4, _, 3, _],
    [1, _, _, 2, _, 2, _, 1]
]

boardSize = len(playground)

# n x n empty Matrix
#board = [[None] * boardSize] * boardSize
board = [[None for _j in range(boardSize)] for _i in range(boardSize)]

# Feed board with Minesweeper Puzzle
for row in range(boardSize):
    for column in range(boardSize):
        if(playground[row][column] is None):
            board[row][column] = solver.IntVar(-1, 0)
        else:
            board[row][column] = solver.IntVar(0, 8)
            solver.Add(board[row][column] == playground[row][column])


# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
for row in range(boardSize):
    for column in range(boardSize):
        if (playground[row][column] is not None):
            rowRange = 0
            columnRange = 0
            rowOffset = 0
            columnOffset = 0
            
            if (row == 0):
                if (column == 0):
                    rowRange = 2
                    columnRange = 2
                elif (column == boardSize - 1):
                    rowRange = 2
                    columnRange = 2
                    rowOffset = 0
                    columnOffset = -1
                else:
                    rowRange = 2
                    columnRange = 3
                    rowOffset = 0
                    columnOffset = -1

                print([playground[row + rowOffset + i][column + columnOffset + j] for i in range(rowRange) for j in range(columnRange)].count(None))
                # solver.Add(solver.Sum([board[row + rowOffset + i][column + columnOffset + j] for i in range(rowRange) for j in range(columnRange)].count()) == 0)
                


# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(np.concatenate(board)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,         # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)       # Value selection policy for search

solver.NewSearch(db)

if solver.NextSolution():
    printSolution(board)
    
solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

2
4
2
Possible solution:
1 -1 -1 -1 2 -1 -1 1 
-1 1 2 -1 3 -1 -1 1 
-1 2 -1 1 -1 -1 -1 0 
-1 2 1 -1 -1 2 3 1 
-1 -1 -1 2 -1 -1 -1 -1 
1 -1 -1 -1 4 3 -1 -1 
-1 1 -1 -1 4 -1 3 -1 
1 -1 -1 2 -1 2 -1 1 
Solutions: 1
Runtime: 2ms
Failures: 0
Branches: 38
